# Importação e preparação dos dados

In [1]:
import pandas as pd

df = pd.read_csv('../dados/acordaos-unicos-filtrados-500.csv', sep = '|')
df['filtrado_500']=df['filtrado_500'].astype(str)
df.head()

,acordao,areas,filtrado_500
0,297/2016-P,Responsabilidade,conta conta gerência instituto nacional seguro...
1,366/2016-P,Finanças Públicas,senado petróleo petróleo regime aduaneiro expo...
2,944/2016-P,Responsabilidade,tribunal conta plenário embargo declaração aco...
3,30/2016-P,Direito Processual,embargo declaração inss recorrente marco antôn...
4,55/2016-P,Pessoal,senac senac senac empresa senac giselle araújo...


In [2]:
df.shape

(9739, 3)

In [3]:
from sklearn.preprocessing import LabelBinarizer

areas = df.groupby(['areas']).groups.keys()
lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
lbArea.classes_

array(['Competência do TCU', 'Contrato Administrativo', 'Convênio',
       'Desestatização', 'Direito Processual', 'Finanças Públicas',
       'Gestão Administrativa', 'Licitação', 'Pessoal',
       'Responsabilidade'], dtype='<U23')

In [4]:
y = lbArea.transform(df['areas'])
y.shape

(9739, 10)

In [5]:
from keras.preprocessing.text import Tokenizer
import numpy as np

vocabulario = 40000
limite_texto = 500
dim_vetor = 100

tokenizer = Tokenizer(num_words=vocabulario)
tokenizer.fit_on_texts(df['filtrado_500'])

sequences = tokenizer.texts_to_sequences(df['filtrado_500'])

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


Found 32293 unique tokens.


In [6]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (9739, 500)


In [7]:
x.shape, y.shape

((9739, 500), (9739, 10))

# Treinamento

In [8]:
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense, Bidirectional
from keras.optimizers import RMSprop
from keras.layers.core import Dropout

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0314 15:39:52.764562 139976409597760 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0314 15:39:52.971354 139976409597760 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0314 15:39:53.022294 139976409597760 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0314 15:39:53.527342 139976409597760 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 100)          4000000   
_________________________________________________________________
gru_1 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2570      
Total params: 4,276,746
Trainable params: 4,276,746
Non-trainable params: 0
_________________________________________________________________


W0314 15:39:54.191352 139976409597760 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 7791 samples, validate on 1948 samples
Epoch 1/20
7791/7791 [==============================] - 114s 15ms/step - loss: 1.4861 - categorical_accuracy: 0.4987 - val_loss: 1.3826 - val_categorical_accuracy: 0.4846
Epoch 2/20
7791/7791 [==============================] - 104s 13ms/step - loss: 1.0826 - categorical_accuracy: 0.6298 - val_loss: 1.3354 - val_categorical_accuracy: 0.5560
Epoch 3/20
7791/7791 [==============================] - 104s 13ms/step - loss: 0.9131 - categorical_accuracy: 0.6939 - val_loss: 1.5115 - val_categorical_accuracy: 0.4420
Epoch 4/20
7791/7791 [==============================] - 103s 13ms/step - loss: 0.7788 - categorical_accuracy: 0.7421 - val_loss: 1.2820 - val_categorical_accuracy: 0.5560
Epoch 5/20
7791/7791 [==============================] - 102s 13ms/step - loss: 0.6636 - categorical_accuracy: 0.7886 - val_loss: 1.4992 - val_categorical_accuracy: 0.5580
Epoch 6/20
7791/7791 [==============================] - 102s 13ms/step - loss: 0.5574 - categoric

In [9]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0314 16:14:17.325772 139976409597760 deprecation.py:506] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 100)          4000000   
_________________________________________________________________
gru_2 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
Total params: 4,276,746
Trainable params: 4,276,746
Non-trainable params: 0
_________________________________________________________________
Train on 7791 samples, validate on 1948 samples
Epoch 1/20
7791/7791 [==============================] - 119s 15ms/step - loss: 1.4830 - categorical_accuracy: 0.4999 - val_loss: 1.4722 - val_categorical_accuracy: 0.4553
Epoch 2/20
7791/7791 [==============================] - 117s 15ms/step - loss: 1.0775 - categorical_accuracy: 0.6380 - val_loss: 1.3827 - val_categorical_accuracy: 0.5390
Epoch 

In [10]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256, dropout=0.1, recurrent_dropout=0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 100)          4000000   
_________________________________________________________________
gru_3 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2570      
Total params: 4,276,746
Trainable params: 4,276,746
Non-trainable params: 0
_________________________________________________________________
Train on 7791 samples, validate on 1948 samples
Epoch 1/20
7791/7791 [==============================] - 120s 15ms/step - loss: 1.5052 - categorical_accuracy: 0.4915 - val_loss: 1.4651 - val_categorical_accuracy: 0.4076
Epoch 2/20
7791/7791 [==============================] - 118s 15ms/step - loss: 1.0986 - categorical_accuracy: 0.6301 - val_loss: 1.3627 - val_categorical_accuracy: 0.4867
Epoch 

In [11]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(512, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 500, 100)          4000000   
_________________________________________________________________
gru_4 (GRU)                  (None, 512)               941568    
_________________________________________________________________
dense_4 (Dense)              (None, 10)                5130      
Total params: 4,946,698
Trainable params: 4,946,698
Non-trainable params: 0
_________________________________________________________________
Train on 7791 samples, validate on 1948 samples
Epoch 1/20
7791/7791 [==============================] - 125s 16ms/step - loss: 1.4953 - categorical_accuracy: 0.5008 - val_loss: 1.5392 - val_categorical_accuracy: 0.5015
Epoch 2/20
7791/7791 [==============================] - 123s 16ms/step - loss: 1.0580 - categorical_accuracy: 0.6450 - val_loss: 1.3206 - val_categorical_accuracy: 0.5693
Epoch 

In [12]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(Bidirectional(GRU(256, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 500, 100)          4000000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               548352    
_________________________________________________________________
dense_5 (Dense)              (None, 10)                5130      
Total params: 4,553,482
Trainable params: 4,553,482
Non-trainable params: 0
_________________________________________________________________
Train on 7791 samples, validate on 1948 samples
Epoch 1/20
7791/7791 [==============================] - 229s 29ms/step - loss: 1.4552 - categorical_accuracy: 0.5230 - val_loss: 1.3981 - val_categorical_accuracy: 0.5010
Epoch 2/20
7791/7791 [==============================] - 227s 29ms/step - loss: 1.0849 - categorical_accuracy: 0.6360 - val_loss: 1.3384 - val_categorical_accuracy: 0.5472
Epoch 